In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Input, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Softmax
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
import scipy.io as scipy
import numpy as np
import pandas as pd
import math
import h5py
import pickle

print(device_lib.list_local_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

## Trainset load

In [ ]:
N_node=4
Bound=1
N=math.comb(N_node,2)
train_data =h5py.File('./trainset.mat','r')
X_train = np.array(train_data['distance'])
Y_train = np.array(train_data['node_position'])

## Validationset load

In [ ]:
N_node=4
Bound=1
N=math.comb(N_node,2)

validation_data =h5py.File('./validationset.mat','r')
X_validation = np.array(validation_data['distance'])
Y_validation = np.array(validation_data['node_position'])

## Train

In [ ]:
input_ = Input(shape=(X_train.shape[1],))

c = Dense(32, input_shape=(X_train.shape[1],)) (input_)
c = Activation('ReLU') (c)

c = Dense(32) (c)
c = Activation('ReLU') (c)

output1 = Dense(1, name='output1')(c)
output2 = Dense(1, name='output2')(c)
output3 = Dense(1, name='output3')(c)
output4 = Dense(1, name='output4')(c)
output5 = Dense(1, name='output5')(c)
output6 = Dense(1, name='output6')(c)
output7 = Dense(1, name='output7')(c)

model = Model(inputs=[input_],outputs=[output1, output2, output3, output4, output5, output6, output7])

model.compile(loss='mse', 
              optimizer=Adagrad(learning_rate=0.01),
              metrics=['mae'])

model.summary()


hist_model = model.fit(X_train, [Y_train[:,0],Y_train[:,1],Y_train[:,2],Y_train[:,3],Y_train[:,4],Y_train[:,5],Y_train[:,6], Y_train[:,8]], batch_size=128, epochs=10, validation_data = (X_validation, 
                [Y_validation[:,0],Y_validation[:,1],Y_validation[:,2],Y_validation[:,3],Y_validation[:,4],Y_validation[:,5],Y_validation[:,6]], Y_validation[:,7]))


model.save('./model.h5') 
with open('./history.h5', 'wb') as file_pi: pickle.dump(hist_model.history, file_pi)


plt.figure(figsize=(5, 5))
plt.subplot(111)           

plt.plot(hist_model.history['loss'],'r')
plt.plot(hist_model.history['val_loss'],'b--')
plt.legend(['Train','Validation'],fontsize = 15)
plt.xlabel('epoch',fontsize = 20)
plt.ylabel('Loss',fontsize = 20)

fig = plt.gcf()